In [1]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
#Initialize SpotiPy with user credentias

spotify = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id = "db0a0f2a96414c5283bb950e7a6cc8f6",
    client_secret = "c7e29a36d7be4cd5981c1bfed88d57c7"))

In [3]:
# creating a variable with the url of the web site we want to scrap

url = "https://www.billboard.com/charts/hot-100"

In [4]:
# crating a variable where we store all the html from the web site using the requests.get() method

response = requests.get(url)

In [5]:
# parse html using beuatifulSoup

billboard = BeautifulSoup(response.content, "html.parser")

---

In [6]:
songs_list = []

for song in billboard.select("span.chart-element__information__song.text--truncate.color--primary"):
    songs_list.append(song.get_text())

---

In [7]:
artist_name = []

for artist in billboard.select("span.chart-element__information__artist.text--truncate.color--secondary"):
    artist_name.append(artist.get_text())

---

In [8]:
current_week_rank = []

for c_week in billboard.select("span.chart-element__rank__number"):
    current_week_rank.append(c_week.get_text())

---

In [9]:
last_week = []

for last in billboard.select("span.chart-element__meta.text--center.color--secondary.text--last"):
    last_week.append(last.get_text())

---

In [10]:
peak_week = []

for peak in billboard.select("span.chart-element__meta.text--center.color--secondary.text--peak"):
    peak_week.append(peak.get_text())

---

In [11]:
weeks = []

for week in billboard.select("span.chart-element__meta.text--center.color--secondary.text--week"):
    weeks.append(week.get_text())

---

In [12]:
billboard_rank = pd.DataFrame({
    "song_name":songs_list,
    "artis":artist_name,
    "current_rank":current_week_rank,
    "last_week":last_week,
    "peak":peak_week,
    "weeks_on_chart":weeks
})

In [13]:
billboard_rank["last_week"].replace(to_replace= "-", value=0, inplace=True)

In [14]:
billboard_rank[['current_rank', 'last_week', 'peak', 'weeks_on_chart']] = billboard_rank[['current_rank', 'last_week', 'peak', 'weeks_on_chart']].astype('int')

In [15]:
billboard_rank["steady"] = np.where(billboard_rank["last_week"]== 0, "New", billboard_rank["last_week"]-billboard_rank["current_rank"])

In [16]:
billboard_rank["song_name"] = billboard_rank["song_name"].apply(lambda x: x.capitalize())

In [17]:
billboard_rank

,song_name,artis,current_rank,last_week,peak,weeks_on_chart,steady
0,Positions,Ariana Grande,1,0,1,1,New
1,Forever after all,Luke Combs,2,0,2,1,New
2,Mood,24kGoldn Featuring iann dior,3,1,1,12,-2
3,Laugh now cry later,Drake Featuring Lil Durk,4,3,2,11,-1
4,Blinding lights,The Weeknd,5,4,1,48,-1
...,...,...,...,...,...,...,...
95,Cardigan,Taylor Swift,96,88,1,14,-8
96,Forget me too,Machine Gun Kelly & Halsey,97,0,44,3,New
97,La toxica,Farruko,98,0,98,1,New
98,Head & heart,Joel Corry X MNEK,99,0,99,1,New


### Playlist Not Hot Songs

In [18]:
playlist = spotify.user_playlist_tracks(user="spotify", playlist_id="spotify:playlist:6yPiKpy7evrwvZodByKvM9")

In [19]:
loops = list(range(0, 10001, 100))

In [20]:
playlist = spotify.user_playlist_tracks(user="spotify", playlist_id="spotify:playlist:6yPiKpy7evrwvZodByKvM9", offset = loops)

In [21]:
loop_list = []

for l in loops:
     loop_list.append(spotify.user_playlist_tracks(user="spotify", playlist_id="spotify:playlist:6yPiKpy7evrwvZodByKvM9", offset = l))
    

In [22]:
len(loop_list)

101

In [23]:
loop_list[0]["items"][0]["track"]["uri"]

'spotify:track:33xMbeHzmWd6Od0BmLZEUs'

In [24]:
#features = []

#for elem in loop_list:
#    for ids in elem["items"]:
#        features.append(ids["track"]["uri"])

In [25]:
#good_indeces = np.array([len(i) for i in features]) <= 36 
#good_list = np.array(features)[good_indeces]
#super_good_indices = np.array([len(i) for i in features]) == 24
#super_good_list = np.array(features)[super_good_indeces]

In [26]:
#audio_features = []

#try: 
#for elem in good_list:
#    audio_features.append(spotify.audio_features(elem))
#except:
#    pass

In [27]:
#len(audio_features)

In [28]:
#audio_features[0]

### Clustering

In [29]:
import pandas as pd
from sklearn import datasets

In [30]:
#features_db.to_csv(r'/Users/pau/Dropbox/Pau/Ironhack/Week3/Project 3/dataframe.csv', index = False, header=True)

In [31]:
features_db = pd.read_csv("data/dataframe.csv")

#pd.concat([pd.DataFrame(ft) for ft in audio_features])

In [32]:
features_db.head()

,0,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
0,NaN,0.0940,https://api.spotify.com/v1/audio-analysis/33xM...,0.310,152560.0,0.445,33xMbeHzmWd6Od0BmLZEUs,0.06780,7.0,0.1130,-13.355,0.0,0.0863,95.360,3.0,https://api.spotify.com/v1/tracks/33xMbeHzmWd6...,audio_features,spotify:track:33xMbeHzmWd6Od0BmLZEUs,0.122
1,NaN,0.1370,https://api.spotify.com/v1/audio-analysis/3Uny...,0.419,197707.0,0.565,3UnyplmZaq547hwsfOR5yy,0.33700,5.0,0.1280,-11.565,1.0,0.0347,151.277,4.0,https://api.spotify.com/v1/tracks/3UnyplmZaq54...,audio_features,spotify:track:3UnyplmZaq547hwsfOR5yy,0.648
2,NaN,0.5900,https://api.spotify.com/v1/audio-analysis/1w8Q...,0.354,244285.0,0.302,1w8QCSDH4QobcQeT4uMKLm,0.07090,9.0,0.1110,-13.078,1.0,0.0326,123.753,4.0,https://api.spotify.com/v1/tracks/1w8QCSDH4Qob...,audio_features,spotify:track:1w8QCSDH4QobcQeT4uMKLm,0.223
3,NaN,0.3250,https://api.spotify.com/v1/audio-analysis/7J9m...,0.727,249940.0,0.785,7J9mBHG4J2eIfDAv5BehKA,0.12600,5.0,0.3100,-6.707,0.0,0.0603,129.649,4.0,https://api.spotify.com/v1/tracks/7J9mBHG4J2eI...,audio_features,spotify:track:7J9mBHG4J2eIfDAv5BehKA,0.960
4,NaN,0.0231,https://api.spotify.com/v1/audio-analysis/1VZe...,0.571,278320.0,0.724,1VZedwJj1gyi88WFRhfThb,0.00311,0.0,0.0919,-5.967,1.0,0.0260,116.755,4.0,https://api.spotify.com/v1/tracks/1VZedwJj1gyi...,audio_features,spotify:track:1VZedwJj1gyi88WFRhfThb,0.385


In [33]:
#duplicating the db so we can remove the columns that contain strings
cluster = features_db.copy(deep=True)

In [34]:
cluster.drop("0", axis = 1, inplace=True)

In [35]:
cluster.drop(["analysis_url", "id", "track_href", "type", "uri"], axis=1, inplace=True)

In [36]:
cluster

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,0.09400,0.310,152560.0,0.445,0.067800,7.0,0.1130,-13.355,0.0,0.0863,95.360,3.0,0.122
1,0.13700,0.419,197707.0,0.565,0.337000,5.0,0.1280,-11.565,1.0,0.0347,151.277,4.0,0.648
2,0.59000,0.354,244285.0,0.302,0.070900,9.0,0.1110,-13.078,1.0,0.0326,123.753,4.0,0.223
3,0.32500,0.727,249940.0,0.785,0.126000,5.0,0.3100,-6.707,0.0,0.0603,129.649,4.0,0.960
4,0.02310,0.571,278320.0,0.724,0.003110,0.0,0.0919,-5.967,1.0,0.0260,116.755,4.0,0.385
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9914,0.12300,0.651,200480.0,0.801,0.000000,0.0,0.0825,-7.340,1.0,0.0513,133.763,4.0,0.776
9915,0.08230,0.811,191560.0,0.647,0.681000,0.0,0.0572,-10.989,1.0,0.0498,139.787,4.0,0.990
9916,0.19000,0.686,206293.0,0.610,0.000931,7.0,0.0710,-5.902,1.0,0.0262,80.567,4.0,0.766
9917,0.00824,0.711,200293.0,0.718,0.002080,6.0,0.0732,-5.739,1.0,0.0380,117.071,4.0,0.688


In [37]:
cluster_clean = cluster.dropna()

In [38]:
from sklearn.preprocessing import StandardScaler

In [39]:
scaler = StandardScaler()

In [40]:
X_scaled = scaler.fit_transform(cluster_clean)

In [41]:
pd.DataFrame(X_scaled)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,-0.625310,-1.622598,-0.924114,-0.738680,-0.193351,0.501942,-0.482243,-1.206798,-1.359244,0.162264,-0.871220,-2.345503,-1.466150
1,-0.489232,-0.936292,-0.400098,-0.226574,0.836020,-0.054942,-0.390614,-0.790320,0.735703,-0.464074,1.076002,0.211118,0.656185
2,0.944334,-1.345557,0.140527,-1.348940,-0.181497,1.058826,-0.494460,-1.142349,0.735703,-0.489565,0.117522,0.211118,-1.058630
3,0.105714,1.002994,0.206164,0.712287,0.029195,-0.054942,0.721144,0.339985,-1.359244,-0.153333,0.322841,0.211118,1.915060
4,-0.849680,0.020758,0.535568,0.451966,-0.440714,-1.447151,-0.611133,0.512160,0.735703,-0.569678,-0.126173,0.211118,-0.404983
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9913,-0.533536,0.524469,-0.367912,0.780567,-0.452606,-1.447151,-0.668554,0.192706,0.735703,-0.262578,0.466104,0.211118,1.172647
9914,-0.662335,1.531891,-0.471446,0.123365,2.151412,-1.447151,-0.823100,-0.656303,0.735703,-0.280786,0.675881,0.211118,2.036106
9915,-0.321508,0.744843,-0.300442,-0.034535,-0.449046,0.501942,-0.738802,0.527284,0.735703,-0.567250,-1.386364,0.211118,1.132298
9916,-0.896706,0.902252,-0.370083,0.426361,-0.444652,0.223500,-0.725363,0.565209,0.735703,-0.424018,-0.115169,0.211118,0.817579


#### Finding the optimal number of clusters

In [42]:
from sklearn.cluster import KMeans

In [43]:
#K = range(2, 10)

In [44]:
#inertia = []

In [45]:
#for i in K:
#        kmeans = KMeans(n_clusters=i,
#                init="random",
#                n_init=100,  # try with 1, 4, 8, 20, 30, 100...
#                max_iter=300,
#                tol=0,
#                algorithm="elkan",
#                random_state=1234)
#        kmeans.fit(X_scaled) 
#        inertia.append(kmeans.inertia_)

In [46]:
#inertia

In [47]:
#import matplotlib.pyplot as plt
#import numpy as np
#%matplotlib inline

#plt.figure(figsize=(16,8))
#plt.plot(K, inertia, 'bx-')
#plt.xlabel('k')
#plt.ylabel('inertia')
#plt.xticks(np.arange(min(K), max(K)+1, 1.0))
#plt.title('Elbow Method showing the optimal k')

In [48]:
#from sklearn.metrics import silhouette_score

In [49]:
#K = range(2, 20)
#silhouette = []

In [50]:
#for i in K:
#    kmeans = KMeans(n_clusters=i, random_state=1234)
        
#    prediction = kmeans.fit_predict(X_scaled)    
#    score=silhouette_score(X_scaled, prediction)
#    silhouette.append(score)

In [51]:
#plt.figure(figsize=(16,8))
#plt.plot(K, silhouette, 'bx-')
#plt.xlabel('k')
#plt.ylabel('silhouette score')
#plt.xticks(np.arange(min(K), max(K)+1, 1.0))
#plt.title('Elbow Method showing the optimal k')

In [52]:
kmeans = KMeans(n_clusters=10)

In [53]:
kmeans.fit(X_scaled)

KMeans(n_clusters=10)

In [54]:
final_clusters = kmeans.predict(X_scaled)

In [55]:
final_clusters

array([2, 6, 2, ..., 5, 5, 3], dtype=int32)

In [56]:
clusters = pd.DataFrame(final_clusters)

In [57]:
features_db["cluster"] = clusters

In [58]:
features_db.drop("0", axis = 1, inplace=True)

In [59]:
features_db.dropna()

,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence,cluster
0,0.09400,https://api.spotify.com/v1/audio-analysis/33xM...,0.310,152560.0,0.445,33xMbeHzmWd6Od0BmLZEUs,0.067800,7.0,0.1130,-13.355,0.0,0.0863,95.360,3.0,https://api.spotify.com/v1/tracks/33xMbeHzmWd6...,audio_features,spotify:track:33xMbeHzmWd6Od0BmLZEUs,0.122,2.0
1,0.13700,https://api.spotify.com/v1/audio-analysis/3Uny...,0.419,197707.0,0.565,3UnyplmZaq547hwsfOR5yy,0.337000,5.0,0.1280,-11.565,1.0,0.0347,151.277,4.0,https://api.spotify.com/v1/tracks/3UnyplmZaq54...,audio_features,spotify:track:3UnyplmZaq547hwsfOR5yy,0.648,6.0
2,0.59000,https://api.spotify.com/v1/audio-analysis/1w8Q...,0.354,244285.0,0.302,1w8QCSDH4QobcQeT4uMKLm,0.070900,9.0,0.1110,-13.078,1.0,0.0326,123.753,4.0,https://api.spotify.com/v1/tracks/1w8QCSDH4Qob...,audio_features,spotify:track:1w8QCSDH4QobcQeT4uMKLm,0.223,2.0
3,0.32500,https://api.spotify.com/v1/audio-analysis/7J9m...,0.727,249940.0,0.785,7J9mBHG4J2eIfDAv5BehKA,0.126000,5.0,0.3100,-6.707,0.0,0.0603,129.649,4.0,https://api.spotify.com/v1/tracks/7J9mBHG4J2eI...,audio_features,spotify:track:7J9mBHG4J2eIfDAv5BehKA,0.960,0.0
4,0.02310,https://api.spotify.com/v1/audio-analysis/1VZe...,0.571,278320.0,0.724,1VZedwJj1gyi88WFRhfThb,0.003110,0.0,0.0919,-5.967,1.0,0.0260,116.755,4.0,https://api.spotify.com/v1/tracks/1VZedwJj1gyi...,audio_features,spotify:track:1VZedwJj1gyi88WFRhfThb,0.385,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9913,0.10900,https://api.spotify.com/v1/audio-analysis/07Cy...,0.541,177987.0,0.830,07CyrZF9eVd02zzIse7tZA,0.701000,9.0,0.1670,-6.246,1.0,0.0316,142.592,4.0,https://api.spotify.com/v1/tracks/07CyrZF9eVd0...,audio_features,spotify:track:07CyrZF9eVd02zzIse7tZA,0.711,5.0
9914,0.12300,https://api.spotify.com/v1/audio-analysis/5jzm...,0.651,200480.0,0.801,5jzma6gCzYtKB1DbEwFZKH,0.000000,0.0,0.0825,-7.340,1.0,0.0513,133.763,4.0,https://api.spotify.com/v1/tracks/5jzma6gCzYtK...,audio_features,spotify:track:5jzma6gCzYtKB1DbEwFZKH,0.776,9.0
9915,0.08230,https://api.spotify.com/v1/audio-analysis/2fmM...,0.811,191560.0,0.647,2fmMPJb5EzZCx8BcNJvVk4,0.681000,0.0,0.0572,-10.989,1.0,0.0498,139.787,4.0,https://api.spotify.com/v1/tracks/2fmMPJb5EzZC...,audio_features,spotify:track:2fmMPJb5EzZCx8BcNJvVk4,0.990,5.0
9916,0.19000,https://api.spotify.com/v1/audio-analysis/60rI...,0.686,206293.0,0.610,60rIdEPDrzyLiLC0icp3xz,0.000931,7.0,0.0710,-5.902,1.0,0.0262,80.567,4.0,https://api.spotify.com/v1/tracks/60rIdEPDrzyL...,audio_features,spotify:track:60rIdEPDrzyLiLC0icp3xz,0.766,5.0


In [60]:
features_db = features_db[features_db['cluster'].notna()]

In [61]:
features_db.cluster = features_db.cluster.apply(lambda x: int(x))

In [62]:
spotify.search(q = playlist["next"])

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=https%3A%2F%2Fapi.spotify.com%2Fv1%2Fplaylists%2F6yPiKpy7evrwvZodByKvM9%2Ftracks%3Foffset%3D100%26limit%3D100%26additional_types%3Dtrack&type=track&offset=0&limit=10',
  'items': [],
  'limit': 10,
  'next': None,
  'offset': 0,
  'previous': None,
  'total': 0}}

---

### Prototipe 1

In [63]:
import random 

In [64]:
# User input function 

def initialize(): 
    
    print("**************************************** Welcome to Gnod's World of Music ****************************************\n\n")
    print("Gnod is a self-adapting system that recomends songs to the users based on the song they input.\nIf the song is currently on the top 100 list Gnods engine recomends another song from the top 100 list.\nIf the song is not on the top 100 list, the powerful Gnods engine will match the features of the song inputed\nand will return a similar song from the Gnods´song database.\n\n\n")
    global song
    global artist
    while True: 
        song = input("Do you have a song in mind?: \n").capitalize()
        print("\n")
        artist = input("Which is the artist/s of your song?: \n(you can say none if you don´t know)\n").capitalize
        print("\n")
        hotSong()
        break

In [65]:
# Check hot song function 
def hotSong(): 
        if song in list(billboard_rank["song_name"]):
            print("\nAwesome, this song is currently on the top 100\n")
            random_100 = random.choice(billboard_rank["song_name"])
            print(f"\nWe believe that you might also like: {random_100}\n")
            moreRecomendations()
        else: 
            not_hot_list()
            #moreRecomendations() 

In [66]:
# recomend song from spotify if the input song is not in the top 100

def not_hot_list():
    global track_id
    if song not in list(billboard_rank["song_name"]):
        track = spotify.search(q = [song, artist],  limit = 1)
        track_id = []
        for t in track["tracks"]["items"]:
            track_id.append(t["uri"])
        audio_features()
        input_scaler = scale_userinput()
        z = match_input(features_db, 'cluster', input_scaler[0])
        y = pd.DataFrame(z)
        p = y['uri'].values
        track_info = p[0]
        u = spotify.track(track_info)
        print("GREAT CHOICE! Here is our recomendation for you:\n")
        print(u['name'], "by", u["album"]["artists"][0]["name"])
        #print(f"The song you have selected is not on the Hot Rank\n\nHowever you might like: {random_alltime}\n\n")

In [67]:
def audio_features():
    global audio_f
    audio_f = spotify.audio_features(track_id)
    #print(audio_features)

In [68]:
def scale_userinput():
    audio_features_df=pd.DataFrame(audio_f)
    audio_features_df=audio_features_df.loc[:,['acousticness', 'danceability', 'duration_ms', 'energy','instrumentalness', 'key', 'liveness', 'loudness', 'mode','speechiness', 'tempo', 'time_signature', 'valence',]]
    X_scaled = scaler.transform(audio_features_df)
    input_scaler = kmeans.predict(X_scaled)
    return input_scaler

In [69]:
def moreRecomendations():
    more = input("Do you want more recomendations? Please enter Yes or No: \n\n").lower()
    while True:
        if more == "yes":
            initialize()
            break
        elif more == "no":
            print("\nThank You, have a good day")
            break

In [70]:
def get_songs():
    songs_list = set()
    
    for song in playlist["items"]:
        songs_list.add(song["track"]["uri"])
        
    print(songs_list)

In [71]:
def match_input(df, cluster_column, input_scaler):
    match = df[df[cluster_column] == input_scaler].sample(1)
    return match

In [77]:
initialize()

**************************************** Welcome to Gnod's World of Music ****************************************


Gnod is a self-adapting system that recomends songs to the users based on the song they input.
If the song is currently on the top 100 list Gnods engine recomends another song from the top 100 list.
If the song is not on the top 100 list, the powerful Gnods engine will match the features of the song inputed
and will return a similar song from the Gnods´song database.





Do you have a song in mind?: 
 Despacito


Which is the artist/s of your song?: 
(you can say none if you don´t know)
 Luis Fonsi




GREAT CHOICE! Here is our recomendation for you:

Careless by Beach Fossils
